In [3]:
#pip install pandas-market-calendars

     |████████████████████████████████| 58 kB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 108 kB 4.6 MB/s eta 0:00:01
  Created wheel for trading-calendars: filename=trading_calendars-2.1.1-py3-none-any.whl size=140919 sha256=ca430fcbc27c1c6d3c50dc2bf8405cc5a1f652e56c52ff96948460fa529aaddf
  Stored in directory: /Users/vigadam/Library/Caches/pip/wheels/f3/a3/ef/b4be7414461eb777966a832a9494924ac12efb80ae4509f54f
Successfully built trading-calendars
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import os
from pathlib import Path
import pandas_market_calendars as mcal
from datetime import datetime
import numpy as np
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [5]:
path = Path(os.getcwd())

In [6]:
base_dir = path.parent.parent

In [7]:
data_in = os.path.join(str(base_dir) , "da_data_repo/swim-transactions/clean/")
data_out = os.path.join("ch18-swim-transactions/")
output = os.path.join(data_out,'output')
#func = os.path.join(str(base_dir) ,   "da_case_studies/ch00-tech-prep/")

In [8]:
daily_agg = pd.read_csv(os.path.join(data_in,"swim_work.csv"),parse_dates=['date'])

In [9]:
daily_agg.head()

,date,QUANTITY
0,2010-01-01,0
1,2010-01-02,49
2,2010-01-03,31
3,2010-01-04,14
4,2010-01-05,18


In [10]:
daily_agg['year']=daily_agg['date'].dt.year
daily_agg['quarter']=daily_agg['date'].dt.quarter
daily_agg['month']=daily_agg['date'].dt.month
daily_agg['day']=daily_agg['date'].dt.day
daily_agg['dow']=daily_agg['date'].dt.dayofweek +1
daily_agg['weekend']=daily_agg['dow'].isin([6,7])


In [11]:
daily_agg.head()

,date,QUANTITY,year,quarter,month,day,dow,weekend
0,2010-01-01,0,2010,1,1,1,5,False
1,2010-01-02,49,2010,1,1,2,6,True
2,2010-01-03,31,2010,1,1,3,7,True
3,2010-01-04,14,2010,1,1,4,1,False
4,2010-01-05,18,2010,1,1,5,2,False


In [12]:
daily_agg.tail()

,date,QUANTITY,year,quarter,month,day,dow,weekend
2552,2016-12-27,12,2016,4,12,27,2,False
2553,2016-12-28,17,2016,4,12,28,3,False
2554,2016-12-29,27,2016,4,12,29,4,False
2555,2016-12-30,3,2016,4,12,30,5,False
2556,2016-12-31,0,2016,4,12,31,6,True


In [13]:
daily_agg['school_off']=(((daily_agg['day']>15) & (daily_agg['month']==5) & (daily_agg['day'] <=30)) | 
           ((daily_agg['month']==6) |  (daily_agg['month']==7)) | 
           ((daily_agg['day']<15) & (daily_agg['month']==8)) | 
           ((daily_agg['day']>20) & (daily_agg['month']==12)))

In [14]:
daily_agg['trend']=daily_agg.index +1 

In [15]:
daily_agg['trend']

0          1
1          2
2          3
3          4
4          5
        ... 
2552    2553
2553    2554
2554    2555
2555    2556
2556    2557
Name: trend, Length: 2557, dtype: int64

In [16]:
nyse = mcal.get_calendar('NYSE')

In [17]:
holidays=nyse.holidays().holidays

In [18]:
daily_agg['isHoliday'] = daily_agg['date'].isin(holidays)

In [19]:
daily_agg.describe()

,QUANTITY,year,quarter,month,day,dow,trend
count,2557.000000,2557.000000,2557.000000,2557.000000,2557.000000,2557.000000,2557.000000
mean,47.867032,2013.000782,2.508408,6.522487,15.730935,4.001173,1279.000000
std,50.491050,2.000587,1.117346,3.449499,8.802361,2.000097,738.286643
min,0.000000,2010.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,14.000000,2011.000000,2.000000,4.000000,8.000000,2.000000,640.000000
50%,28.000000,2013.000000,3.000000,7.000000,16.000000,4.000000,1279.000000
75%,64.000000,2015.000000,4.000000,10.000000,23.000000,6.000000,1918.000000
max,287.000000,2016.000000,4.000000,12.000000,31.000000,7.000000,2557.000000


In [20]:
daily_agg['q_month'] = daily_agg.groupby('month')['QUANTITY'].transform('mean')

In [21]:
daily_agg['QUANTITY2'] = np.where(daily_agg['QUANTITY']<1, 1,daily_agg['QUANTITY'])

In [22]:
daily_agg['q_ln'] = np.log(daily_agg['QUANTITY2'])

In [23]:
daily_agg['tickets'] = daily_agg.groupby(['month','dow'])['QUANTITY'].transform('mean')

In [24]:
daily_agg['tickets_ln'] = daily_agg.groupby(['month','dow'])['q_ln'].transform('mean')

In [25]:
daily_agg['dow_abb'] = daily_agg['date'].dt.day_name().str[:3]

In [26]:
daily_agg['month_abb'] = daily_agg['date'].dt.month_name().str[:3]

In [27]:
### Creat train/holdout data

In [28]:
data_holdout = daily_agg.loc[daily_agg['year']==2016,:]

In [29]:
data_train = daily_agg.loc[daily_agg['year']<2016,:]

In [30]:
list(map(len,[daily_agg,data_holdout,data_train]))

[2557, 366, 2191]

In [31]:
data_train.tail()

,date,QUANTITY,year,quarter,month,day,dow,weekend,school_off,trend,isHoliday,q_month,QUANTITY2,q_ln,tickets,tickets_ln,dow_abb,month_abb
2186,2015-12-27,0,2015,4,12,27,7,True,True,2187,False,15.97235,1,0.000000,25.266667,2.927724,Sun,Dec
2187,2015-12-28,16,2015,4,12,28,1,False,True,2188,False,15.97235,16,2.772589,13.483871,2.392609,Mon,Dec
2188,2015-12-29,32,2015,4,12,29,2,False,True,2189,False,15.97235,32,3.465736,14.612903,2.507322,Tue,Dec
2189,2015-12-30,15,2015,4,12,30,3,False,True,2190,False,15.97235,15,2.708050,14.903226,2.470384,Wed,Dec
2190,2015-12-31,16,2015,4,12,31,4,False,True,2191,False,15.97235,16,2.772589,13.093750,2.321463,Thu,Dec


In [32]:
factor_cols = ['month', 'dow', 'isHoliday', 'school_off']
data_train[factor_cols] = data_train[factor_cols].astype('category')

/usr/local/lib/python3.8/site-packages/pandas/core/frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [33]:
data_train.dtypes

date          datetime64[ns]
QUANTITY               int64
year                   int64
quarter                int64
month               category
day                    int64
dow                 category
weekend                 bool
school_off          category
trend                  int64
isHoliday           category
q_month              float64
QUANTITY2              int64
q_ln                 float64
tickets              float64
tickets_ln           float64
dow_abb               object
month_abb             object
dtype: object

In [34]:
import patsy

In [35]:
y,X = patsy.dmatrices("QUANTITY ~ 1+ trend + month",data_train)

In [36]:
X

DesignMatrix with shape (2191, 13)
  Columns:
    ['Intercept',
     'month[T.2]',
     'month[T.3]',
     'month[T.4]',
     'month[T.5]',
     'month[T.6]',
     'month[T.7]',
     'month[T.8]',
     'month[T.9]',
     'month[T.10]',
     'month[T.11]',
     'month[T.12]',
     'trend']
  Terms:
    'Intercept' (column 0)
    'month' (columns 1:12)
    'trend' (column 12)
  (to view full data, use np.asarray(this_obj))

In [37]:
#X = data_train.loc[:,['trend','month']].to_numpy()
#y = data_train.loc[:,'QUANTITY'].to_numpy()
groups = data_train.loc[:,'year'].to_numpy()

In [38]:
lin_reg = LinearRegression(fit_intercept=False)

In [39]:
logo = LeaveOneGroupOut()

In [40]:
y_true=[]
y_pred=[]
rmse_folds=[]
for train_index, test_index in logo.split(X, y, groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lin_reg.fit(X_train,y_train)
    y_hat = lin_reg.predict(X_test)
    y_true.append(y_test)
    y_pred.append(y_hat)
    rmse_folds.append(mean_squared_error(y_test,y_hat,squared=False))
#mean_squared_error(np.concatenate(y_true).ravel(),np.concatenate(y_pred).ravel(),squared=False)

In [41]:
sum(rmse_folds)/len(rmse_folds)

32.347871356433025

In [149]:
y,X = patsy.dmatrices("QUANTITY ~ 1+ trend + month + dow",data_train)

In [154]:
rmse_folds=[]
for train_index, test_index in logo.split(X, y, groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lin_reg.fit(X_train,y_train)
    y_hat = lin_reg.predict(X_test)
    rmse_folds.append(mean_squared_error(y_test,y_hat,squared=False))

In [155]:
sum(rmse_folds)/len(rmse_folds)

31.448481901741218

In [156]:
# creating instance of one-hot-encoder
enc = OneHotEncoder(handle_unknown='ignore')
# passing bridge-types-cat column (label encoded values of bridge_types)
enc_df = pd.DataFrame(enc.fit_transform(data_train[['month']]).toarray())

In [157]:
enc_df_d = enc_df.drop(columns=0)

In [158]:
data_train = data_train.join(enc_df_d)

ValueError: columns overlap but no suffix specified: Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], dtype='object')

In [159]:
data_train.columns

Index([      'date',   'QUANTITY',       'year',    'quarter',      'month',
              'day',        'dow',    'weekend', 'school_off',      'trend',
        'isHoliday',    'q_month',  'QUANTITY2',       'q_ln',    'tickets',
       'tickets_ln',    'dow_abb',  'month_abb',            1,            2,
                  3,            4,            5,            6,            7,
                  8,            9,           10,           11],
      dtype='object')

In [160]:
X = data_train.loc[:,['trend',1,2,3,4,5,6,7,8,9,10,11]].to_numpy()

In [161]:
data_train.shape

(2191, 29)

In [162]:
X.shape

(2191, 12)

In [163]:
lin_reg=LinearRegression(fit_intercept=True)

In [164]:
reg1 = lin_reg.fit(X[train_list[0][0]],y[train_list[0][0]])

In [165]:
reg1.coef_

array([[-9.22296369e-04,  5.95123925e-01,  1.78223419e+01,
         1.67560633e+01,  4.20656987e+01,  1.22358990e+02,
         8.66574427e+01,  3.81440984e+01,  3.81050800e+00,
        -5.86738309e-01, -3.36656526e+00, -3.82725242e+00]])

In [166]:
lin_reg.predict(X_test[train_list[0][1]])

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 12 is different from 19)

In [167]:
len(y_test)

365

In [168]:
len(np.concatenate(y_pred).ravel())

2191

In [169]:
for train_index, test_index in logo.split(X, y, groups):
    print("TRAIN:", train_index, "TEST:", test_index)
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]
print(X_train, X_test, y_train, y_test)

TRAIN: [ 365  366  367 ... 2188 2189 2190] TEST: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 2